# 01 - Exploratory Data Analysis of RAW input data

In [1]:
# Import necessary libraries
import rasterio
from rasterio.enums import Resampling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from skimage.feature import graycomatrix, graycoprops
from rasterio.warp import reproject, calculate_default_transform
from typing import Tuple, Dict, Any

# Set plotting style
sns.set(style="whitegrid")

In [ ]:
# Utils functions
def plot_image(image: np.ndarray, title: str, cmap: str = 'viridis') -> None:
    """Plot a single band image.

    Args:
        image (np.ndarray): The image array to plot.
        title (str): Title of the plot.
        cmap (str): Colormap to use for the plot.
    """
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap=cmap)
    plt.colorbar()
    plt.title(title)
    plt.axis('off')
    plt.show()

def plot_histogram(image: np.ndarray, title: str) -> None:
    """Plot a histogram of pixel values in an image.

    Args:
        image (np.ndarray): The image array to plot.
        title (str): Title of the histogram.
    """
    plt.figure(figsize=(10, 6))
    plt.hist(image.flatten(), bins=50, color='c', edgecolor='k', alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Values')
    plt.ylabel('Frequency')
    plt.show()

